In [34]:
!pip install py2neo

In [35]:
from py2neo import Graph

In [36]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [37]:
type(graph)

py2neo.database.Graph

In [1]:
import pandas as pd
df_empl = pd.read_csv("employees.csv") #employee info
df_act = pd.read_csv("actors.csv") #actor info
df_hobby = pd.read_csv("hobby.csv") #character hobby info
df_hob_pop = pd.read_csv("hobby_popularity.csv") #hobby popularity
df_jobs = pd.read_csv("jobs.csv") #character dunder mif job
df_jobs


,id,job,salary,typical educational requirement,percentile,salary_cat
0,1,salesperson,"40,000",bachelor's,45,medium
1,2,receptionist,"30,000",high school diploma,34,medium
2,3,CEO,"250,000",bachelor's,97,high
3,4,HR,"50,000",bachelor's,55,medium
4,5,dock worker,"30,000",high school diploma,34,medium
5,6,regional manager,"60,000",bachelor's,64,medium
6,7,accountant,"45,000",bachelor's,50,medium
7,8,customer service,"35,000",high school diploma,40,medium
8,9,temp,"15,000",bachelor's,15,low
9,10,quality assurance,"50,000",bachelor's,55,medium


In [42]:
graph.run("MATCH (n) DETACH DELETE n")
from py2neo import Node
list = []

# populate db with all office character
for i in range(0,len(df_empl)):
    person = Node("Person", name = df_empl['Employee_Name'].iloc[i], branch = df_empl['last_branch'].iloc[i], episodes = (df_empl['episodes'].iloc[i]).item())
    graph.create(person)
    
# populate db with all office jobs
for i in range(0, len(df_jobs)):
    job = Node("Job", name = (df_jobs['job'].iloc[i]).title(), salary = df_jobs['salary'].iloc[i], education = df_jobs['typical_educational_requirement'].iloc[i])
    graph.create(job)


In [98]:
from py2neo import Relationship
WorksAs = Relationship.type("WORKS_AS")

from py2neo import Graph, NodeMatcher
#graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
matcher = NodeMatcher(graph)

for i in range(0,len(df_empl)):
    empl_job = df_jobs[df_jobs['id'] == df_empl["job_ID"].iloc[i].item()][["job"]].iloc[0][0].title()
    per = df_empl["Employee_Name"].iloc[i]
    per_node = matcher.match("Person").where(name=per).first()
    empl_node = matcher.match("Job").where(name=empl_job).first()
    graph.create(WorksAs(per_node, empl_node))
        

'Michael Scott'

Node('Job', education="bachelor's", name='Regional Manager', salary='60,000')

"_.name = 'Michael Scott'"
